In [156]:
#from util import *
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [157]:
import scipy.io as scio
from scipy import signal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_path = "/Users/n.y.r/Desktop/bcidata/datasets_matlab/"
electrodeID_path = '/Users/n.y.r/Desktop/BCI_Project/metadata/' #path to git repo CSV files

all_data = {}
all_data['N'] = {i: scio.loadmat(data_path + 'i140703-001_raw-lfp-spikes_ch{}.mat'.format(str(i).zfill(2))) for i in range(1,97)}
all_data['L'] = {i: scio.loadmat(data_path + 'l101210-001_raw-lfp-spikes_ch{}.mat'.format(str(i).zfill(2))) for i in range(1,97)}

def get_trial_time_and_label(monkey, event_want='GO-ON'):
    assert monkey in ['L', 'N']
    data = all_data[monkey][1]
    event_data = data['block']['segments'][0][0][0][0][0][0]['events'][0][2][0][0]
    assert event_data['name'] == 'TrialEvents'
    assert event_data['times_units'] == '(1.0/30000 * s)'
    samples = []
    t_trial = None
    lb_trial = None
    valid_trial = True
    valid_trial2 = False
    for event, label, time in zip(event_data['an_trial_event_labels'],
                            event_data['an_belongs_to_trialtype'],
                            event_data['times'][0]):
        if event.strip() == event_want.strip():
            t_trial = time/30000
            lb_trial = label
        if 'ERROR' in event:
            valid_trial = False
        if 'RW-ON' in event:
            valid_trial2 = True
        if event[:5] in ['TS-ON', 'NONE ']:
            if valid_trial and valid_trial2 and lb_trial:
                samples.append((t_trial, lb_trial))
            t_trial = None
            lb_trial = None
            valid_trial = True
    return samples

def get_electrodes(monkey_name, x, y, step):
    chunk_channel_id=[]
    assert monkey_name in ['L', 'N']
    if monkey_name=='L':
        df=pd.read_csv(electrodeID_path+'L.csv')
    else:
        df=pd.read_csv(electrodeID_path+'N.csv')   
    monkey_electrodes_id=df['Id'].tolist()
    start=(y-1)*10+x
    for col in range (0, step):
        for row in range (0,step):
            chunk_channel_id.append(monkey_electrodes_id[start+10*row-1])
        start=start+1   
    return chunk_channel_id

def get_st_feat(monkey, ch, start_time, window_time, unit_id_flag):
    data = all_data[monkey][ch]
    tmp = data['block']['segments'][0][0][0][0][0][0]['spiketrains'][0]
    st = None
    if unit_id_flag=='MIN':
        #print('1111111111111111111111')
        for i in range(tmp.shape[0]):
            if tmp[i][0][0]['an_unit_id'].item() == 1: st = tmp[i][0][0]
            elif tmp[i][0][0]['an_unit_id'].item() == 0 and st is None: st = tmp[i][0][0]
    if unit_id_flag=='MAX':
        #print('****************')
        if tmp[tmp.shape[0]-1][0][0]['an_unit_id'].item() != 255:
            st = tmp[tmp.shape[0]-1][0][0]
        else: 
            st = tmp[tmp.shape[0]-2][0][0]
    if unit_id_flag=='ZERO':
        st = tmp[0][0][0]
    if unit_id_flag=='Total':
        for i in range(tmp.shape[0]):
            if tmp[i][0][0]['an_unit_id'].item() != 255: st.append(tmp[i][0][0])
            
        
    assert st['times_units'].item() == '(1.0/30000 * s)'
    st['times'][0]
    st_op = start_time*30000
    st_ed = (start_time+window_time)*30000

    frs = []
    parts = 10
    for i in range(parts):
        sop = st_op + (st_ed-st_op)*i/parts
        sed = st_op + (st_ed-st_op)*(i+1)/parts
        frs.append(((st['times'][0] >= sop) & (st['times'][0] < sed)).sum())
    # return [((st['times'][0] >= st_op) & (st['times'][0] < st_ed)).sum()]
    return frs

def get_lfp_feat(monkey, ch, start_time, window_time):
    data = all_data[monkey][ch]
    lfp_data = data['block']['segments'][0][0][0][0][0][0]['analogsignals'][0][0][0][0]#['file_origin'][0][-3:]
    assert lfp_data['file_origin'][0][-3:] == 'ns2'
    assert lfp_data['t_start_units'] == '(1.0/1000.0*s)'
    start_time_shifted = start_time - lfp_data['t_start'].item()
    
    
    psds = []
    parts = 10
    for i in range(parts):
        op = int((start_time_shifted+window_time*i/parts)*1000)
        ed = int((start_time_shifted+window_time*(i+1)/parts)*1000)
        sig = lfp_data['signal'].squeeze()[op:ed]
        freqs, psd = signal.welch(sig, 1000) #nperseg=500)
        psds.append(psd[:])
    return np.concatenate(psds)

def get_all_feat_and_labels(monkey, event_want, offset, window_time,unit_id_flag):
    trials = get_trial_time_and_label(monkey, event_want)

    all_feat = {}

    for ch in range(1,97):
        st_feat = []
        if monkey == 'N': lfp_feat = []
        for trial_t, label in trials:
            start_time = trial_t + offset
            st_feat.append(get_st_feat(monkey, ch, start_time, window_time,unit_id_flag))
            if monkey == 'N': lfp_feat.append(get_lfp_feat(monkey, ch, start_time, window_time))
        all_feat[ch] = {'st_feat': np.stack(st_feat)}
        if monkey == 'N': all_feat[ch]['lfp_feat'] = np.stack(lfp_feat)
    labels = np.stack([trial[1] for trial in trials])
    
    return all_feat, labels

def plot_heatmap(accs, vmin=0.25, vmax=1):
    cols, rows = accs.shape
    plt.imshow(accs, interpolation='nearest', extent=[0.5, 0.5+cols, 0.5, 0.5+rows],
                      cmap='bwr', vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.axis('off')
    plt.show()

In [158]:
def run_single_time_window(monkey, event_want, offset, time_window, unit_id_flag,  spatial_step=3):
    spatial_xid=[]
    spatial_yid=[]
    acc_ls=[]
    time_id=[]
    all_feat, labels = get_all_feat_and_labels(monkey, event_want, offset, time_window,unit_id_flag)
    #labels = labels.view('<U2')[::2]
    for i in range (1, 10-spatial_step+2):
        for j in range (1, 10-spatial_step+2): 
            elecs = get_electrodes(monkey, i, j, spatial_step)
            data = []
            for ch in elecs:
                if ch == -1: continue
                data.append(all_feat[ch]['st_feat'])
            if monkey == 'N':
                for ch in elecs:
                    if ch == -1: continue
                    data.append(all_feat[ch]['lfp_feat'])
            data = np.concatenate(data, axis=1)
            train_num=int(0.8*data.shape[0])

            X_train, y_train, X_test, y_test = data[:train_num], labels[:train_num], data[train_num:], labels[train_num:]
            # Normalize data
            scaler = StandardScaler()  # normalization: zero mean, unit variance
            scaler.fit(X_train)  # scaling factor determined from the training set

            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
            
           # pca = PCA(n_components=100)
           # pca.fit(X_train)
          #  X_train = pca.transform(X_train)
            #X_test = pca.transform(X_test)
            
            
#             clf = MLPClassifier(hidden_layer_sizes=(50), learning_rate_init=0.01, max_iter=500, verbose=0)
            clf = SVC(kernel='sigmoid', C=1)
            clf.fit(X_train, y_train)

            accuracy = (clf.predict(X_test) == y_test).mean()
            spatial_xid.append(i)
            spatial_yid.append(j)
            acc_ls.append(accuracy)
            time_id.append(offset)
    return pd.DataFrame(list(zip(spatial_xid, spatial_yid, time_id, acc_ls)), columns = ['X', 'Y', 'Time Stamp', 'Accuracy'])

In [159]:
accs = []
for _ in tqdm(range(10)):
    df = run_single_time_window('L', 'SR', -0.8, 1.6,'Total')
    accs.append(df['Accuracy'].mean())
np.mean(accs)

  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'append'

In [155]:
np.mean(accs)

0.46145833333333314

In [148]:
accs

[]

In [149]:
df['Accuracy'].mean()

0.4614583333333332

In [164]:
xlist=[]
ylist=[]
for i in range (1, 9):
    for j in range (1, 9):
        xlist.append(j)
        ylist.append(i)
dfdf=pd.DataFrame(list(zip(ylist, xlist)), columns = ['X', 'Y'])
dfdf.to_csv('/Users/n.y.r/Desktop/N_spatial.csv')

In [167]:
df_L=pd.read_csv('/Users/n.y.r/Desktop/L_spatial.csv')

In [170]:
df_L['Motor_Status']=df_L['Motor_Status'].replace(to_replace = np.nan, value = 1)

In [171]:
df_L

,Unnamed: 0,X,Y,Motor_Status
0,0,1,1,0.0
1,1,1,2,0.0
2,2,1,3,0.0
3,3,1,4,0.0
4,4,1,5,2.0
...,...,...,...,...
59,59,8,4,1.0
60,60,8,5,1.0
61,61,8,6,1.0
62,62,8,7,1.0


In [172]:
df_N=pd.read_csv('/Users/n.y.r/Desktop/N_spatial.csv')
df_N['Motor_Status']=df_N['Motor_Status'].replace(to_replace = np.nan, value = 1)

In [173]:
df_N

,Unnamed: 0,X,Y,Motor_Status
0,0,1,1,0.0
1,1,1,2,0.0
2,2,1,3,0.0
3,3,1,4,0.0
4,4,1,5,1.0
...,...,...,...,...
59,59,8,4,1.0
60,60,8,5,1.0
61,61,8,6,1.0
62,62,8,7,1.0


In [175]:
N_original=pd.read_csv('/Users/n.y.r/Downloads/BCI_Project/N6.csv')
N_original

,Unnamed: 0,index,X,Y,Spatial Step,Event,Time Start,Time Span,Accuracy
0,0,0,1,1,3,SR,-0.8,0.1,0.548387
1,1,1,1,2,3,SR,-0.8,0.1,0.645161
2,2,2,1,3,3,SR,-0.8,0.1,0.580645
3,3,3,1,4,3,SR,-0.8,0.1,0.774194
4,4,4,1,5,3,SR,-0.8,0.1,0.483871
...,...,...,...,...,...,...,...,...,...
1979,1979,59,8,4,3,SR,0.6,0.2,0.806452
1980,1980,60,8,5,3,SR,0.6,0.2,0.806452
1981,1981,61,8,6,3,SR,0.6,0.2,0.774194
1982,1982,62,8,7,3,SR,0.6,0.2,0.806452


In [181]:
df_test_n=N_original.merge(df_N, on='Unnamed: 0', how='left')
df_test_n

,Unnamed: 0,index,X_x,Y_x,Spatial Step,Event,Time Start,Time Span,Accuracy,X_y,Y_y,Motor_Status
0,0,0,1,1,3,SR,-0.8,0.1,0.548387,1.0,1.0,0.0
1,1,1,1,2,3,SR,-0.8,0.1,0.645161,1.0,2.0,0.0
2,2,2,1,3,3,SR,-0.8,0.1,0.580645,1.0,3.0,0.0
3,3,3,1,4,3,SR,-0.8,0.1,0.774194,1.0,4.0,0.0
4,4,4,1,5,3,SR,-0.8,0.1,0.483871,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1979,1979,59,8,4,3,SR,0.6,0.2,0.806452,NaN,NaN,NaN
1980,1980,60,8,5,3,SR,0.6,0.2,0.806452,NaN,NaN,NaN
1981,1981,61,8,6,3,SR,0.6,0.2,0.774194,NaN,NaN,NaN
1982,1982,62,8,7,3,SR,0.6,0.2,0.806452,NaN,NaN,NaN


TypeError: unsupported type: <class 'str'>

In [183]:
dict_N=df_N.set_index('Unnamed: 0','').to_dict()['Motor_Status']

In [184]:
dict_N

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 1.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 1.0,
 13: 1.0,
 14: 1.0,
 15: 1.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 1.0,
 20: 1.0,
 21: 1.0,
 22: 1.0,
 23: 1.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 1.0,
 28: 1.0,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 0.0,
 33: 0.0,
 34: 1.0,
 35: 1.0,
 36: 1.0,
 37: 1.0,
 38: 1.0,
 39: 1.0,
 40: 0.0,
 41: 0.0,
 42: 1.0,
 43: 1.0,
 44: 1.0,
 45: 1.0,
 46: 1.0,
 47: 1.0,
 48: 0.0,
 49: 1.0,
 50: 1.0,
 51: 1.0,
 52: 1.0,
 53: 1.0,
 54: 1.0,
 55: 1.0,
 56: 0.0,
 57: 1.0,
 58: 1.0,
 59: 1.0,
 60: 1.0,
 61: 1.0,
 62: 1.0,
 63: 1.0}

In [189]:
df_N['New_Spatial'] = df_N['X'].astype(str) + "_" +  df_N['Y'].astype(str)

In [190]:
df_N

,Unnamed: 0,X,Y,Motor_Status,NEW_SPATIAL,New_Spatial
0,0,1,1,0.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,1_1
1,1,1,2,0.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,1_2
2,2,1,3,0.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,1_3
3,3,1,4,0.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,1_4
4,4,1,5,1.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,1_5
...,...,...,...,...,...,...
59,59,8,4,1.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,8_4
60,60,8,5,1.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,8_5
61,61,8,6,1.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,8_6
62,62,8,7,1.0,[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3...,8_7


In [191]:
dict_N=df_N.set_index('New_Spatial').to_dict()['Motor_Status']

In [192]:
dict_N

{'1_1': 0.0,
 '1_2': 0.0,
 '1_3': 0.0,
 '1_4': 0.0,
 '1_5': 1.0,
 '1_6': 1.0,
 '1_7': 1.0,
 '1_8': 1.0,
 '2_1': 0.0,
 '2_2': 0.0,
 '2_3': 0.0,
 '2_4': 0.0,
 '2_5': 1.0,
 '2_6': 1.0,
 '2_7': 1.0,
 '2_8': 1.0,
 '3_1': 0.0,
 '3_2': 0.0,
 '3_3': 0.0,
 '3_4': 1.0,
 '3_5': 1.0,
 '3_6': 1.0,
 '3_7': 1.0,
 '3_8': 1.0,
 '4_1': 0.0,
 '4_2': 0.0,
 '4_3': 0.0,
 '4_4': 1.0,
 '4_5': 1.0,
 '4_6': 1.0,
 '4_7': 1.0,
 '4_8': 1.0,
 '5_1': 0.0,
 '5_2': 0.0,
 '5_3': 1.0,
 '5_4': 1.0,
 '5_5': 1.0,
 '5_6': 1.0,
 '5_7': 1.0,
 '5_8': 1.0,
 '6_1': 0.0,
 '6_2': 0.0,
 '6_3': 1.0,
 '6_4': 1.0,
 '6_5': 1.0,
 '6_6': 1.0,
 '6_7': 1.0,
 '6_8': 1.0,
 '7_1': 0.0,
 '7_2': 1.0,
 '7_3': 1.0,
 '7_4': 1.0,
 '7_5': 1.0,
 '7_6': 1.0,
 '7_7': 1.0,
 '7_8': 1.0,
 '8_1': 0.0,
 '8_2': 1.0,
 '8_3': 1.0,
 '8_4': 1.0,
 '8_5': 1.0,
 '8_6': 1.0,
 '8_7': 1.0,
 '8_8': 1.0}

In [193]:
N_original['New_Spatial'] = N_original['X'].astype(str) + "_" +  N_original['Y'].astype(str)

In [195]:
N_original['Motor_Status']= N_original['New_Spatial'].map(dict_N)
N_original.head(10)


,Unnamed: 0,index,X,Y,Spatial Step,Event,Time Start,Time Span,Accuracy,New_Spatial,Motor_Status
0,0,0,1,1,3,SR,-0.8,0.1,0.548387,1_1,0.0
1,1,1,1,2,3,SR,-0.8,0.1,0.645161,1_2,0.0
2,2,2,1,3,3,SR,-0.8,0.1,0.580645,1_3,0.0
3,3,3,1,4,3,SR,-0.8,0.1,0.774194,1_4,0.0
4,4,4,1,5,3,SR,-0.8,0.1,0.483871,1_5,1.0
5,5,5,1,6,3,SR,-0.8,0.1,0.580645,1_6,1.0
6,6,6,1,7,3,SR,-0.8,0.1,0.612903,1_7,1.0
7,7,7,1,8,3,SR,-0.8,0.1,0.516129,1_8,1.0
8,8,8,2,1,3,SR,-0.8,0.1,0.516129,2_1,0.0
9,9,9,2,2,3,SR,-0.8,0.1,0.516129,2_2,0.0


In [197]:
N_original=N_original.drop(['Unnamed: 0', 'New_Spatial'],axis=1)

In [198]:
N_original.to_csv('/Users/n.y.r/Desktop/N6_MotorStatus.csv')

In [199]:
L_original=pd.read_csv('/Users/n.y.r/Downloads/BCI_Project/L6.csv')
L_original['New_Spatial'] = L_original['X'].astype(str) + "_" +  L_original['Y'].astype(str)
df_L=pd.read_csv('/Users/n.y.r/Desktop/L_spatial.csv')
df_L['Motor_Status']=df_L['Motor_Status'].replace(to_replace = np.nan, value = 1)
df_L['New_Spatial'] = df_L['X'].astype(str) + "_" +  df_L['Y'].astype(str)
dict_L=df_L.set_index('New_Spatial').to_dict()['Motor_Status']
L_original['Motor_Status']= L_original['New_Spatial'].map(dict_L)
L_original=L_original.drop(['Unnamed: 0', 'New_Spatial'],axis=1)
L_original['Motor_Status'].value_counts()

1.0    1519
0.0     310
2.0     155
Name: Motor_Status, dtype: int64

In [200]:
L_original.head(10)

,index,X,Y,Spatial Step,Event,Time Start,Time Span,Accuracy,Motor_Status
0,0,1,1,3,SR,-0.8,0.1,0.800000,0.0
1,1,1,2,3,SR,-0.8,0.1,0.800000,0.0
2,2,1,3,3,SR,-0.8,0.1,0.700000,0.0
3,3,1,4,3,SR,-0.8,0.1,0.533333,0.0
4,4,1,5,3,SR,-0.8,0.1,0.633333,2.0
5,5,1,6,3,SR,-0.8,0.1,0.666667,1.0
6,6,1,7,3,SR,-0.8,0.1,0.700000,1.0
7,7,1,8,3,SR,-0.8,0.1,0.700000,1.0
8,8,2,1,3,SR,-0.8,0.1,0.733333,0.0
9,9,2,2,3,SR,-0.8,0.1,0.733333,0.0


In [201]:
L_original.to_csv('/Users/n.y.r/Desktop/L6_MotorStatus.csv')